In [42]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy.cosmology import LambdaCDM 
from astropy import cosmology
#from astroquery.simbad import Simbad
#from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [86]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [96]:
# reading in the relevant tables here
#liu2011 = ((Table.read('Tables/XLiu2011b/table1.dat', readme = 'Tables/XLiu2011b/ReadMe', format='ascii.cds')).to_pandas())
#print(len(xliu2011))

#name_to_coords(xliu2011,xliu2011['SDSS'])

# loading in the N-tuples from the Liu 2011 sample
liu2011triples = pd.read_csv('liu2011_triples.csv', sep=',')
liu2011quad = pd.read_csv('liu2011_quadruples.csv', sep=',')
liu2011quint = pd.read_csv('liu2011_quintuple.csv', sep=',')
liu2011quint['Literature Name'] = "SDSS J0959+1259 (CG)"
# selection stregies has been programmatically fixed down helow.

# adding triples from Tytler+2009 
tyt2009 = pd.read_csv('Tytler_triples_table_DR1.csv', sep=',')
# selection stregies has been programmatically fixed down helow.

# loading in the triples from the literature
littriples = pd.read_csv('MAC_individualtriples_singleline.csv', sep=',')
#selection stregies fixed in the table manually

# loading in the quadruples from the literature
litquads = pd.read_csv('MAC_individualquadruples_singleline.csv', sep=',')
#**** Double check the separations
#selection stregies fixed in the table manually

# loading in the quintuples from the literature
litquints = pd.read_csv('MAC_individualquintuples_singleline.csv', sep=',')
#**** check the separations
# this is the same the liu2011 quint so we'll defer to that table instead below

# loading in the sextuplet from the literature
litsix = pd.read_csv('MAC_individualsextuples_singleline.csv', sep=',')
#**** check separations
#selection stregies fixed in the table manually




In [97]:
#littriples

In [98]:
# Here we're going to add the relevant columns for the Liu+2011 multiplets
liu2011triples['System Type 1: Triple AGNs'] = 'Triple AGN Candidate'
liu2011triples['System Type 2: Dual AGNs'] = 'Dual AGN Candidate'
liu2011triples['System Type 3: Recoils'] = '-99'
liu2011triples['ST1 Confidence Flag'] = 0.5
liu2011triples['ST2 Confidence Flag'] = 0.5
liu2011triples['ST3 Confidence Flag'] = -99
liu2011triples['ST1 Num AGNs'] = 3
liu2011triples['ST2 Num AGNs'] = 2
liu2011triples['ST3 Num AGNs'] = -99
liu2011triples['Selection Method'] = "{'Optical':['Spectroscopy','Fiber Spectroscopy','Spectroscopic Emission Line Ratios']}"
liu2011triples['Confirmation Method'] = '-99'
liu2011triples['Paper(s)'] = 'Liu+2011b'
liu2011triples['BibCode(s)'] = '2011ApJ...737..101L'
liu2011triples['DOI(s)'] = 'https://doi.org/10.1088/0004-637X/737/2/101'

#'ST1 Num AGNs','ST2 Num AGNs','ST3 Num AGNs'
# running through and assigning information for the triples from the Liu 2011 sample
for i in [1,2,3]:
    liu2011triples['z'+str(i)] = -99.0
    liu2011triples['z'+str(i)+"_type"] = "spec"
    liu2011triples['Equinox'+str(i)] = "J2000"
    liu2011triples['Coordinate_waveband'+str(i)] = "Optical"
    liu2011triples['Coordinate_Source'+str(i)] = "SDSS"
    liu2011triples['AGN_type'+str(i)] = -99
    for index, row in liu2011.iterrows():
        liu2011triples.loc[liu2011triples['Name'+str(i)]==row['SDSS'], 'z'+str(i)] = row['z']
        liu2011triples.loc[liu2011triples['Name'+str(i)]==row['SDSS'], 'AGN_type'+str(i)] = row['FAGN']

# running through this exercise now for the quadruples
liu2011quad['System Type 1: Quadruple AGNs'] = 'Quadruple AGN Candidate'
liu2011quad['System Type 2: Triple AGNs'] = 'Triple AGN Candidate'
liu2011quad['System Type 3: Dual AGNs'] = 'Dual AGN Candidate'
liu2011quad['ST1 Confidence Flag'] =0.5
liu2011quad['ST2 Confidence Flag'] = 0.5
liu2011quad['ST3 Confidence Flag'] = 0.5
#ST1 Num AGNs,ST2 Num AGNs,ST3 Num AGNs
# I'm adding in the Num AGN columns here.... just note that we may need to come back in the future and update some of this
liu2011quad['ST1 Num AGNs'] = 4
liu2011quad['ST2 Num AGNs'] = 3
liu2011quad['ST3 Num AGNs'] = 2
liu2011quad['Selection Method'] = "{'Optical':['Spectroscopy','Fiber Spectroscopy','Spectroscopic Emission Line Ratios']}"
liu2011quad['Confirmation Method'] = '-99'
liu2011quad['Paper(s)'] = 'Liu+2011b'
liu2011quad['BibCode(s)'] = '2011ApJ...737..101L'
liu2011quad['DOI(s)'] = 'https://doi.org/10.1088/0004-637X/737/2/101'

for i in [1,2,3,4]:
    liu2011quad['z'+str(i)] = -99.0
    liu2011quad['z'+str(i)+"_type"] = "spec"
    liu2011quad['Equinox'+str(i)] = "J2000"
    liu2011quad['Coordinate_waveband'+str(i)] = "Optical"
    liu2011quad['Coordinate_Source'+str(i)] = "SDSS"
    liu2011quad['AGN_type'+str(i)] = -99
    for index, row in liu2011.iterrows():
        liu2011quad.loc[liu2011quad['Name'+str(i)]==row['SDSS'], 'z'+str(i)] = row['z']
        liu2011quad.loc[liu2011quad['Name'+str(i)]==row['SDSS'], 'AGN_type'+str(i)] = row['FAGN']

# aaaaand now for the quintuple from Liu+2011 (note, this object was also the one De Rosa 2015 looked at)
liu2011quint['System Type 1: Quintuple AGNs'] = 'Quintuple AGN Candidate'
liu2011quint['System Type 2: Triple AGNs'] = 'Triple AGN Candidate'
liu2011quint['System Type 3: Dual AGNs'] = 'Dual AGN Candidate'
liu2011quint['ST1 Confidence Flag'] = 0.5
liu2011quint['ST2 Confidence Flag'] = 0.5
liu2011quint['ST3 Confidence Flag'] = 0.5
liu2011quint['ST1 Num AGNs'] = 5
liu2011quint['ST2 Num AGNs'] = 3
liu2011quint['ST3 Num AGNs'] = 2
liu2011quint['Selection Method'] = "{'Optical':['Spectroscopy','Fiber Spectroscopy','Spectroscopic Emission Line Ratios'],'X-ray':['Imaging','Spectroscopy']}"
liu2011quint['Confirmation Method'] = '-99'
liu2011quint['Paper(s)'] = 'Liu+2011b ; De Rosa+2015'
liu2011quint['BibCode(s)'] = '2011ApJ...737..101L ; 2015MNRAS.453..214D'
liu2011quint['DOI(s)'] = 'https://doi.org/10.1088/0004-637X/737/2/101 ; https://doi.org/10.1093/mnras/stv1623'

for i in [1,2,3,4,5]:
    liu2011quint['z'+str(i)] = -99.0
    liu2011quint['z'+str(i)+"_type"] = "spec"
    liu2011quint['Equinox'+str(i)] = "J2000"
    liu2011quint['Coordinate_waveband'+str(i)] = "Optical"
    liu2011quint['Coordinate_Source'+str(i)] = "SDSS"
    liu2011quint['AGN_type'+str(i)] = -99
    for index, row in liu2011.iterrows():
        liu2011quint.loc[liu2011quint['Name'+str(i)]==row['SDSS'], 'z'+str(i)] = row['z']
        liu2011quint.loc[liu2011quint['Name'+str(i)]==row['SDSS'], 'AGN_type'+str(i)] = row['FAGN']


#xliu_agns = xliu[((xliu['FAGN1']==0) | (xliu['FAGN1']==3) | (xliu['FAGN1']==4) | (xliu['FAGN1']==5)) & \
#                 ((xliu['FAGN2']==0) | (xliu['FAGN2']==3) | (xliu['FAGN2']==4) | (xliu['FAGN2']==5)) & \
#                 (xliu['Sep']>10)]
#
#for index, row in gandalf.iterrows():
#    if row['Name1'] in xliu_agns_1list:
#        #print('True')
#        gandalf.at[index, 'Confidence Flag'] = 1 
#        gandalf.at[index, 'Confirmation Method'] = gandalf.at[index, 'Selection Method']
#        gandalf.at[index, 'Primary System Type'] = 'Dual AGN'
#    elif (row['Paper(s)']=='Liu+2011b') and (row['Name1'] not in xliu_agns_1list):
#        gandalf.at[index, 'Confidence Flag'] = +0.5
#    elif (row['Paper(s)']=='Liu+2011b ; Barrows+2016') and (row['Name1'] not in xliu_agns_1list):
#        gandalf.at[index, 'Confidence Flag'] = +0.5


In [99]:
# we need to do some formatting for the lit triples table here
# SDSS J1056+5516A is going to have a recoil possibility as the third system type. But that means the third system type for all of the \
# triples needs to be recoils


#littriples

In [100]:
tyt2009['System Type 1: Triple AGNs'] = 'Triple AGN Candidate'
tyt2009['System Type 2: Dual AGNs'] = '-99'
tyt2009['System Type 3: Recoilings'] = '-99'

tyt2009['ST1 Confidence Flag'] = -1
tyt2009['ST2 Confidence Flag'] = -99
tyt2009['ST3 Confidence Flag'] = -99

tyt2009['ST1 Num AGNs'] = 3
tyt2009['ST2 Num AGNs'] = -99
tyt2009['ST3 Num AGNs'] = -99

tyt2009['Selection Method'] = "{'Optical':['Spectroscopy']}"
tyt2009['Confirmation Method'] = '-99'

tyt2009['z1'] = tyt2009['zem1']
tyt2009['z2'] = tyt2009['zem2']
tyt2009['z3'] = tyt2009['zem3']

for index, row in tyt2009.iterrows():
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011triples.at[index, 'Sep23'] = c1.separation(c2).arcsecond


for index, row in tyt2009.iterrows():
    tyt2009.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)

#tyt2009
#littriples
#litquads
#litquints
#litsix
#liu2011triples

# and based on the separations alone in the tytler 2009 triples table, we can flag them as 'unlikely' triples

In [101]:
# Now adding in relevant columns for the literature triples table
#littriples['System Type 3: Single AGNs'] = "-99"
#littriples['ST3 Confidence Rank'] = -99

##littriples['ST1 Num AGNs'] = 3
#littriples['ST2 Num AGNs'] = 3
#littriples['ST3 Num AGNs'] = 2

for index, row in littriples.iterrows():
    try:
        c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
        c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
        littriples.at[index, 'Sep12'] = c1.separation(c2).arcsecond
        c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
        c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
        littriples.at[index, 'Sep13'] = c1.separation(c2).arcsecond
        c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
        c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
        littriples.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    except: 
        print('Coordinates not available for angular separation calculation.')
        
for index, row in littriples.iterrows():
    try:
        littriples.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    except:
        print('Angular separation and/or redshift not available for physical separation calculation.')
        
# we'll need to rewrite the separations for the Roland+2020 target (we're going to overwrite it when calculating \
# separations)

# we need coordinates for Djogovski+ and Schawinski+

#littriples


Coordinates not available for angular separation calculation.
Coordinates not available for angular separation calculation.


In [102]:
#(10/55)*(93)


In [103]:
#(1/65)*(25)

In [104]:
# now calculating the angular and physical separations for all N-tuples in the Liu+2011 sample
for index, row in liu2011triples.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011triples.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011triples.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011triples.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    
for index, row in liu2011triples.iterrows():
    liu2011triples.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
 
    
for index, row in liu2011quad.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep14'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep24'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep34'] = c1.separation(c2).arcsecond
    
for index, row in liu2011quad.iterrows():
    liu2011quad.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)14_z1'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)14_z4'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)24_z2'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)24_z3'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)34_z3'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)34_z4'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
 


In [105]:
# and still calculating these for the Liu+2011 sample, but here for the quintuple
for index, row in liu2011quint.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep14'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep15'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep24'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep25'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep34'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep35'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep45'] = c1.separation(c2).arcsecond
    

for index, row in liu2011quint.iterrows():
    liu2011quint.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)14_z1'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)14_z4'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)15_z1'] = row['Sep15']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)15_z5'] = row['Sep15']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    #
    liu2011quint.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)24_z2'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)24_z4'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)25_z2'] = row['Sep25']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)25_z5'] = row['Sep25']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    #
    liu2011quint.at[index, 'Sep(kpc)34_z3'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)34_z4'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)35_z3'] = row['Sep35']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)35_z5'] = row['Sep35']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    #
    liu2011quint.at[index, 'Sep(kpc)45_z4'] = row['Sep45']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)45_z5'] = row['Sep45']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)



In [106]:
#liu2011quad

In [107]:
# Adding relevant columns for the AGN quadruple from Hennawi+2015
litquads['System Type 1: Quadruple AGNs'] = 'Quadruple AGN Candidate'
litquads['System Type 2: Triple AGNs'] = '-99'
litquads['System Type 3: Dual AGNs'] = '-99'
litquads['ST1 Confidence Flag'] = 1
litquads['ST2 Confidence Flag'] = -99
litquads['ST3 Confidence Flag'] = -99
#litquads['Selection Method'] = 'Optical Spectroscopy / Fiber Spectroscopy / Spectroscopic Emission Line Ratios'
#litquads['Confirmation Method'] = '-99'

# calculating physical and angular separations for the literature quadruples
for index, row in litquads.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep14'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep24'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep34'] = c1.separation(c2).arcsecond

for index, row in litquads.iterrows():
    litquads.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)14_z1'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)14_z4'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)24_z2'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)24_z3'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)34_z3'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)34_z4'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)




In [108]:
# adding relevant columns for the sextuplet
litsix['System Type 1: Sextuple AGNs'] = 'Sextuplet AGN Candidate'
litsix['System Type 2: Triple AGNs'] = '-99'
litsix['System Type 3: Dual AGNs'] = '-99'
litsix['ST1 Confidence Flag'] = -1 # here we are flagging it as -1
# NOT because they're not AGNs in a dense environment/protocluster, but because these are NOT the result of \
# mergers and their activity is probably dominated by the large scale environment. 
# Specifically: the physical separations are all >200 kpc between the various AGNs. 
litsix['ST2 Confidence Flag'] = -99
litsix['ST3 Confidence Flag'] = -99
#litsix['Selection Method'] = 'Optical Spectroscopy / Fiber Spectroscopy / Spectroscopic Emission Line Ratios'
#litsix['Confirmation Method'] = '-99'

# calculating physical and angular separations now for the literature sextuplet
for index, row in litsix.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep14'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep15'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep16'] = c1.separation(c2).arcsecond    
    
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep24'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep25'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep26'] = c1.separation(c2).arcsecond
    
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep34'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep35'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep36'] = c1.separation(c2).arcsecond
    
    c1 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep45'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep46'] = c1.separation(c2).arcsecond
    
    c1 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep56'] = c1.separation(c2).arcsecond

    

for index, row in litsix.iterrows():
    litsix.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)14_z1'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)14_z4'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)15_z1'] = row['Sep15']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)15_z5'] = row['Sep15']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)16_z1'] = row['Sep16']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)16_z6'] = row['Sep16']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)
    #
    litsix.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)24_z2'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)24_z4'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)25_z2'] = row['Sep25']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)25_z5'] = row['Sep25']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)26_z2'] = row['Sep26']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)26_z6'] = row['Sep26']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)    
    #
    litsix.at[index, 'Sep(kpc)34_z3'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)34_z4'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)35_z3'] = row['Sep35']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)35_z5'] = row['Sep35']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)36_z3'] = row['Sep36']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)36_z6'] = row['Sep36']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)
    #
    litsix.at[index, 'Sep(kpc)45_z4'] = row['Sep45']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)45_z5'] = row['Sep45']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)46_z4'] = row['Sep46']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)46_z6'] = row['Sep46']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)
    #
    litsix.at[index, 'Sep(kpc)56_z5'] = row['Sep56']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)56_z6'] = row['Sep56']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)



In [109]:
#litquads

In [110]:
#liu2011triples

In [111]:
#litsix

In [112]:
#liu2011

In [113]:
# And now here we are collecting all of our information into a few distinct tables

MACtriples = pd.concat([liu2011triples,littriples]).reset_index(drop=True)
MACquads = pd.concat([liu2011quad,litquads]).reset_index(drop=True)

# the other tables do not require any additional concatenation

# and now we're going to generate a 'bad' table that will also be uploaded
badtriples = tyt2009.copy()
# as far as I know, only the tytler targets are bad among the triples


In [114]:
#MACtriples


In [115]:
# adding here the velocity difference information for all of the tables that require it

MACtriples['dV_12'] = -99.99
MACtriples['dV_13'] = -99.99
MACtriples['dV_23'] = -99.99
for index, row in MACtriples.iterrows():
    for i in [12,13,23]:
        try:
        #    #print('z'+str(i)[0],'z'+str(i)[1])
        #    MACtriples.at[index, 'dV_'+str(i)] = (2.99e+5) * ((1 + row['z'+str(i)[0]])**2 - (1 + row['z'+str(i)[1]])**2) / ((1 + row['z'+str(i)[0]])**2 + (1 + row['z'+str(i)[1]])**2)
        #except:
        #    print('Velocity difference cannot be calculated.')
        #    print(row['z'+str(i)[0]])
        #    MACtriples.at[index, 'dV_'+str(i)] = -99.99
            # Calculate only if both z values are present and not zero
            if pd.notna(row['z'+str(i)[0]]) and pd.notna(row['z'+str(i)[1]]) and row['z'+str(i)[0]] != 0 and row['z'+str(i)[1]] != 0:
                MACtriples.at[index, 'dV_'+str(i)] = (2.99e+5) * (
                    (1 + row['z'+str(i)[0]])**2 - (1 + row['z'+str(i)[1]])**2) / ((1 + row['z'+str(i)[0]])**2 + (1 + row['z'+str(i)[1]])**2)
            else:
                raise ValueError(f"Invalid or zero z values: z"+str(i)[0]+f"={row['z'+str(i)[0]]}, z"+str(i)[1]+f"={row['z'+str(i)[1]]}")
        except Exception as e:
            print(f'Error for index {index}, pair {i}: {e}')
            MACtriples.at[index, 'dV_'+str(i)] = -99.99

for index, row in MACquads.iterrows():
    for i in [12,13,14,23,24,34]:
        try:
            if pd.notna(row['z'+str(i)[0]]) and pd.notna(row['z'+str(i)[1]]) and row['z'+str(i)[0]] != 0 and row['z'+str(i)[1]] != 0:
                MACquads.at[index, 'dV_'+str(i)] = (2.99e+5) * (
                    (1 + row['z'+str(i)[0]])**2 - (1 + row['z'+str(i)[1]])**2) / ((1 + row['z'+str(i)[0]])**2 + (1 + row['z'+str(i)[1]])**2)
            else:
                raise ValueError(f"Invalid or zero z values: z"+str(i)[0]+f"={row['z'+str(i)[0]]}, z"+str(i)[1]+f"={row['z'+str(i)[1]]}")
        except Exception as e:
            print(f'Error for index {index}, pair {i}: {e}')
            MACquads.at[index, 'dV_'+str(i)] = -99.99

for index, row in liu2011quint.iterrows():
    for i in [12,13,14,15,23,24,25,34,35,45]:
        try:
            if pd.notna(row['z'+str(i)[0]]) and pd.notna(row['z'+str(i)[1]]) and row['z'+str(i)[0]] != 0 and row['z'+str(i)[1]] != 0:
                liu2011quint.at[index, 'dV_'+str(i)] = (2.99e+5) * (
                    (1 + row['z'+str(i)[0]])**2 - (1 + row['z'+str(i)[1]])**2) / ((1 + row['z'+str(i)[0]])**2 + (1 + row['z'+str(i)[1]])**2)
            else:
                raise ValueError(f"Invalid or zero z values: z"+str(i)[0]+f"={row['z'+str(i)[0]]}, z"+str(i)[1]+f"={row['z'+str(i)[1]]}")
        except Exception as e:
            print(f'Error for index {index}, pair {i}: {e}')
            liu2011quint.at[index, 'dV_'+str(i)] = -99.99

for index, row in litsix.iterrows():
    for i in [12,13,14,15,16,23,24,25,26,34,35,36,45,46,56]:
        try:
            if pd.notna(row['z'+str(i)[0]]) and pd.notna(row['z'+str(i)[1]]) and row['z'+str(i)[0]] != 0 and row['z'+str(i)[1]] != 0:
                litsix.at[index, 'dV_'+str(i)] = (2.99e+5) * (
                    (1 + row['z'+str(i)[0]])**2 - (1 + row['z'+str(i)[1]])**2) / ((1 + row['z'+str(i)[0]])**2 + (1 + row['z'+str(i)[1]])**2)
            else:
                raise ValueError(f"Invalid or zero z values: z"+str(i)[0]+f"={row['z'+str(i)[0]]}, z"+str(i)[1]+f"={row['z'+str(i)[1]]}")
        except Exception as e:
            print(f'Error for index {index}, pair {i}: {e}')
            Mlitsix.at[index, 'dV_'+str(i)] = -99.99

    

Error for index 41, pair 12: Invalid or zero z values: z1=1.355, z2=nan
Error for index 41, pair 13: Invalid or zero z values: z1=1.355, z3=nan
Error for index 41, pair 23: Invalid or zero z values: z2=nan, z3=nan
Error for index 52, pair 12: Invalid or zero z values: z1=0.11111, z2=nan
Error for index 52, pair 13: Invalid or zero z values: z1=0.11111, z3=nan
Error for index 52, pair 23: Invalid or zero z values: z2=nan, z3=nan


In [116]:
# dropping miscellaneous columns in the triples, quads, and quints tables
MACtriples.drop(['Sep12(kpc)','Sep13(kpc)','Sep23(kpc)'], axis=1, inplace=True)
MACquads.drop(['System Type','Sep12(kpc)','Sep13(kpc)','Sep14(kpc)'], axis=1, inplace=True)

badtriples.drop(['index','zem1','---1','Pair','m_Pair1','zem2','---2','m_Pair2','zem3','---3','m_Pair3',\
                 'Table_flag'], axis=1, inplace=True)


In [117]:
#MACtriples.columns
#MACquads.columns
#badtriples.columns
liu2011quint.columns
#litsix.columns

Index(['Name1', 'RA1', 'Dec1', 'Name2', 'RA2', 'Dec2', 'Name3', 'RA3', 'Dec3',
       'Name4', 'RA4', 'Dec4', 'Name5', 'RA5', 'Dec5', 'Literature Name',
       'System Type 1: Quintuple AGNs', 'System Type 2: Triple AGNs',
       'System Type 3: Dual AGNs', 'ST1 Confidence Flag',
       'ST2 Confidence Flag', 'ST3 Confidence Flag', 'ST1 Num AGNs',
       'ST2 Num AGNs', 'ST3 Num AGNs', 'Selection Method',
       'Confirmation Method', 'Paper(s)', 'BibCode(s)', 'DOI(s)', 'z1',
       'z1_type', 'Equinox1', 'Coordinate_waveband1', 'Coordinate_Source1',
       'AGN_type1', 'z2', 'z2_type', 'Equinox2', 'Coordinate_waveband2',
       'Coordinate_Source2', 'AGN_type2', 'z3', 'z3_type', 'Equinox3',
       'Coordinate_waveband3', 'Coordinate_Source3', 'AGN_type3', 'z4',
       'z4_type', 'Equinox4', 'Coordinate_waveband4', 'Coordinate_Source4',
       'AGN_type4', 'z5', 'z5_type', 'Equinox5', 'Coordinate_waveband5',
       'Coordinate_Source5', 'AGN_type5', 'Sep12', 'Sep13', 'Sep14', 'Sep15',


In [118]:
# here we're dropping the brightness columns from this data release
MACtriples.drop(['Brightness_band1', 'Brightness_type1', 'Brightness2',\
       'Brightness_band2', 'Brightness_type2', 'Brightness3',\
       'Brightness_band3', 'Brightness_type3', 'Notes','Brightness1',\
        'AGN_type1','AGN_type2','AGN_type3'], axis=1, inplace=True)
MACquads.drop(['Notes','AGN_type2','AGN_type1','AGN_type3','AGN_type4'], axis=1, inplace=True)
#liu2011quints.drop(['System Type','Notes','Brightness3', 'Brightness_band3',\
#       'Brightness_type3','Brightness2', 'Brightness_band2', 'Brightness_type2',\
#        'Brightness_band1', 'Brightness_type1','Brightness4',\
#       'Brightness_band4', 'Brightness_type4','Brightness5', 'Brightness_band5', 'Brightness_type5'], axis=1, inplace=True)

In [119]:
# we need to take a closer look at the individual objects (Farina for example is a dual AGN but probably not \
# a triple AGN because AGN 1 is ~200 kpc away from the other two)

#litquints


In [120]:
#liu2011quint
#Serendipitous":[], "Optical":["Narrow Band Imaging"]},{"Optical": ["Spectroscopy"]}

In [121]:
# rearranging the columns for these dataframes now to put them int he correct format
# triples
MACtriples = MACtriples[['System Type 1: Triple AGNs', 'System Type 2: Dual AGNs',\
       'System Type 3: Recoils', 'ST1 Confidence Flag', 'ST2 Confidence Flag',\
       'ST3 Confidence Flag', 'ST1 Num AGNs', 'ST2 Num AGNs', 'ST3 Num AGNs','Literature Name','Selection Method', \
       'Confirmation Method','Name1', 'RA1', 'Dec1', 'z1', 'z1_type', 'Equinox1', 'Coordinate_waveband1',\
       'Coordinate_Source1','Name2', 'RA2', 'Dec2', 'z2', 'z2_type', 'Equinox2',\
       'Coordinate_waveband2', 'Coordinate_Source2','Name3', 'RA3', 'Dec3', 'z3', 'z3_type',\
       'Equinox3', 'Coordinate_waveband3', 'Coordinate_Source3', 'Sep12',\
       'Sep13', 'Sep23', 'Sep(kpc)12_z1', 'Sep(kpc)12_z2', 'Sep(kpc)13_z1',\
       'Sep(kpc)13_z3', 'Sep(kpc)23_z2', 'Sep(kpc)23_z3',\
       'dV_12', 'dV_13', 'dV_23','Paper(s)', 'BibCode(s)','DOI(s)']]


MACquads = MACquads[['System Type 1: Quadruple AGNs','System Type 2: Triple AGNs', 'System Type 3: Dual AGNs',\
       'ST1 Confidence Flag', 'ST2 Confidence Flag', 'ST3 Confidence Flag',\
       'ST1 Num AGNs', 'ST2 Num AGNs', 'ST3 Num AGNs','Literature Name','Selection Method',\
       'Confirmation Method', 'Name1', 'RA1', 'Dec1', 'z1',\
       'z1_type', 'Equinox1', 'Coordinate_waveband1', 'Coordinate_Source1',\
       'Name2', 'z2', 'z2_type', 'RA2', 'Dec2', 'Equinox2', 'Coordinate_waveband2','Coordinate_Source2', \
       'Name3', 'z3', 'z3_type', 'RA3', 'Dec3','Equinox3','Coordinate_waveband3', 'Coordinate_Source3',\
       'Name4',  'z4', 'z4_type','RA4', 'Dec4','Equinox4', 'Coordinate_waveband4', 'Coordinate_Source4', \
       'Sep12','Sep13', 'Sep14', 'Sep23', 'Sep24', 'Sep34', 'Sep(kpc)12_z1',\
       'Sep(kpc)12_z2', 'Sep(kpc)13_z1', 'Sep(kpc)13_z3', 'Sep(kpc)23_z2',\
       'Sep(kpc)23_z3', 'Sep(kpc)14_z1', 'Sep(kpc)14_z4', 'Sep(kpc)24_z2',\
       'Sep(kpc)24_z3', 'Sep(kpc)34_z3', 'Sep(kpc)34_z4', \
       'dV_12', 'dV_13', 'dV_14', 'dV_23', 'dV_24', 'dV_34','Paper(s)', 'BibCode(s)', 'DOI(s)']]


liu2011quint = liu2011quint[['System Type 1: Quintuple AGNs', 'System Type 2: Triple AGNs',
       'System Type 3: Dual AGNs', 'ST1 Confidence Flag', 'ST2 Confidence Flag', 'ST3 Confidence Flag',\
       'ST1 Num AGNs','ST2 Num AGNs', 'ST3 Num AGNs','Literature Name','Selection Method','Confirmation Method',\
       'Name1', 'RA1', 'Dec1', 'z1','z1_type', 'Equinox1', 'Coordinate_waveband1', 'Coordinate_Source1',\
       'Name2', 'RA2', 'Dec2', 'z2', 'z2_type', 'Equinox2', 'Coordinate_waveband2',\
       'Coordinate_Source2','Name3', 'RA3', 'Dec3','z3', 'z3_type', 'Equinox3',\
       'Coordinate_waveband3', 'Coordinate_Source3', 'Name4', 'RA4', 'Dec4','z4',\
       'z4_type', 'Equinox4', 'Coordinate_waveband4', 'Coordinate_Source4',\
       'Name5', 'RA5', 'Dec5','z5', 'z5_type', 'Equinox5', 'Coordinate_waveband5','Coordinate_Source5',\
       'Sep12', 'Sep13', 'Sep14', 'Sep15','Sep23', 'Sep24', 'Sep25', 'Sep34', 'Sep35', 'Sep45', 'Sep(kpc)12_z1',\
       'Sep(kpc)12_z2', 'Sep(kpc)13_z1', 'Sep(kpc)13_z3', 'Sep(kpc)14_z1',\
       'Sep(kpc)14_z4', 'Sep(kpc)15_z1', 'Sep(kpc)15_z5', 'Sep(kpc)23_z2',\
       'Sep(kpc)23_z3', 'Sep(kpc)24_z2', 'Sep(kpc)24_z4', 'Sep(kpc)25_z2',\
       'Sep(kpc)25_z5', 'Sep(kpc)34_z3', 'Sep(kpc)34_z4', 'Sep(kpc)35_z3',\
       'Sep(kpc)35_z5', 'Sep(kpc)45_z4', 'Sep(kpc)45_z5', 'dV_12', 'dV_13',\
       'dV_14', 'dV_15', 'dV_23', 'dV_24', 'dV_25', 'dV_34', 'dV_35', 'dV_45','Paper(s)', 'BibCode(s)', 'DOI(s)']]


In [122]:
MACtriples
MACquads

,System Type 1: Quadruple AGNs,System Type 2: Triple AGNs,System Type 3: Dual AGNs,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,ST1 Num AGNs,ST2 Num AGNs,ST3 Num AGNs,Literature Name,Selection Method,Confirmation Method,Name1,RA1,Dec1,z1,z1_type,Equinox1,Coordinate_waveband1,Coordinate_Source1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Name3,z3,z3_type,RA3,Dec3,Equinox3,Coordinate_waveband3,Coordinate_Source3,Name4,z4,z4_type,RA4,Dec4,Equinox4,Coordinate_waveband4,Coordinate_Source4,Sep12,Sep13,Sep14,Sep23,Sep24,Sep34,Sep(kpc)12_z1,Sep(kpc)12_z2,Sep(kpc)13_z1,Sep(kpc)13_z3,Sep(kpc)23_z2,Sep(kpc)23_z3,Sep(kpc)14_z1,Sep(kpc)14_z4,Sep(kpc)24_z2,Sep(kpc)24_z3,Sep(kpc)34_z3,Sep(kpc)34_z4,dV_12,dV_13,dV_14,dV_23,dV_24,dV_34,Paper(s),BibCode(s),DOI(s)
0,Quadruple AGN Candidate,Triple AGN Candidate,Dual AGN Candidate,0.5,0.5,0.5,4.0,3.0,2.0,NaN,"{'Optical':['Spectroscopy','Fiber Spectroscopy...",-99,J135632.71+232118.3,13:56:32.71,+23:21:18.3,0.0661,spec,J2000,Optical,SDSS,J135635.73+232136.0,0.0668,spec,13:56:35.73,+23:21:36.0,J2000,Optical,SDSS,J135638.42+232151.5,0.0659,spec,13:56:38.42,+23:21:51.5,J2000,Optical,SDSS,J135639.45+232210.3,0.0657,spec,13:56:39.45,+23:22:10.3,J2000,Optical,SDSS,45.197554,85.351350,106.385871,40.154178,61.647560,23.549803,57.308866,57.868275,108.222428,107.920278,51.411034,50.771898,134.893442,134.140033,78.929889,77.730300,29.776931,29.693523,-196.258594,56.097560,112.205644,252.356123,308.464162,56.108087,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101
1,Quadruple AGN Candidate,Triple AGN Candidate,Dual AGN Candidate,0.5,0.5,0.5,4.0,3.0,2.0,NaN,"{'Optical':['Spectroscopy','Fiber Spectroscopy...",-99,J144702.08+134006.2,14:47:02.08,+13:40:06.2,0.0304,spec,J2000,Optical,SDSS,J144705.78+133953.1,0.0307,spec,14:47:05.78,+13:39:53.1,J2000,Optical,SDSS,J144708.49+133917.0,0.0285,spec,14:47:08.49,+13:39:17.0,J2000,Optical,SDSS,J144712.81+133939.3,0.0323,spec,14:47:12.81,+13:39:39.3,J2000,Optical,SDSS,55.496917,105.592377,158.690871,53.511475,103.391141,66.799826,33.762481,34.083393,64.238894,60.361504,32.864035,30.589643,96.542253,102.342708,63.497597,66.678879,38.185881,43.080456,-87.040899,551.847604,-550.830967,638.888160,-463.790317,-1102.674822,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101
2,Quadruple AGN Candidate,-99,-99,1.0,-99.0,-99.0,NaN,NaN,NaN,Jackpot Nebula,"{'Serendipitous':[], 'Optical':['Imaging','Nar...",{'Optical': ['Spectroscopy']},J084158.47+392121.0 (f/g QSO),08:41:58.47,+39:21:21.0,2.0412,spec,J2000,Optical,SDSS,J084158.24+392129.1 (AGN1),2.0550,spec,08:41:58.24,+39:21:29.1,J2000,Ly-alpha at redshift,KECK/LRIS,J084158.66+392114.7 (AGN2),2.0580,spec,08:41:58.66,+39:21:14.7,J2000,Ly-alpha at redshift,Keck/LRIS,J084159.10+392104.6 (AGN3),2.0500,spec,08:41:59.10,+39:21:04.6,J2000,Ly-alpha at redshift,Keck/LRIS,8.527952,6.674305,17.954249,15.201626,26.452701,11.316170,71.220522,71.162570,55.739938,55.684605,126.851886,126.829163,149.943511,149.865983,220.737904,220.803442,94.412298,94.457249,-1353.688808,-1647.154345,-863.933054,-293.472856,489.762161,783.233758,Hennawi+2015,2015Sci...348..779H,https://doi.org/2015Sci...348..779H


In [123]:
# and here we're saving the tables for the repository

MACtriples.to_csv('MAC_tripleAGNs.csv', sep=',', index=False)
MACquads.to_csv('MAC_quadrupleAGNs.csv', sep=',', index=False)
liu2011quint.to_csv('MAC_quintupleAGN.csv', sep=',', index=False)
# note, the sextuplet will be listed in the catalog and online, but as an 'unlikely' system as far as \
# interaction driven. Hence, why the file name has 'bad' abended to it
litsix.to_csv('MAC_sextupleAGN_bad.csv', sep=',', index=False)
badtriples.to_csv('MAC_Ntuples_bad.csv', sep=',', index=False)




In [71]:
## and now for calculating the physical separations.....
#
#for i in [1,2,3,4,5]:
#    liu2011quad['z'+str(i)] = -99
#    liu2011quad['z'+str(i)+"_type"] = "spec"
#    liu2011quad['Equinox'+str(i)] = "J2000"
#    liu2011quad['Coordinate_waveband'+str(i)] = "Optical"
#    liu2011quad['Coordinate_Source'+str(i)] = "SDSS"
#    liu2011quad['AGN_type'+str(i)] = -99
#    for index, row in liu2011.iterrows():
#        liu2011quad.loc[liu2011quad['Name'+str(i)]==row['SDSS'], 'z'+str(i)] = row['z']
#        liu2011quad.loc[liu2011quad['Name'+str(i)]==row['SDSS'], 'AGN_type'+str(i)] = row['FAGN']
#
#
#for index, row in gandalf_white.iterrows():
#    #gandalf_white.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
#    gandalf_white.at[index, 'Sep(kpc)12_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
#    gandalf_white.at[index, 'Sep(kpc)12_z2'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
#
#
#
#for index, row in gandalf_white.iterrows():
#    #gandalf_white.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
#    gandalf_white.at[index, 'Sep(kpc)_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
#    gandalf_white.at[index, 'Sep(kpc)_z2'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
#


In [42]:
#xliu_agns_1list = xliu_agns['Name1'].to_list()
#xliu_agns_2list = xliu_agns['Name2'].to_list()
#
#for index, row in gandalf.iterrows():
#    if row['Name1'] in xliu_agns_1list:
#        #print('True')
#        gandalf.at[index, 'Confidence Flag'] = 1 
#        gandalf.at[index, 'Confirmation Method'] = gandalf.at[index, 'Selection Method']
#        gandalf.at[index, 'Primary System Type'] = 'Dual AGN'
#    elif (row['Paper(s)']=='Liu+2011b') and (row['Name1'] not in xliu_agns_1list):
#        gandalf.at[index, 'Confidence Flag'] = +0.5
#    elif (row['Paper(s)']=='Liu+2011b ; Barrows+2016') and (row['Name1'] not in xliu_agns_1list):
#        gandalf.at[index, 'Confidence Flag'] = +0.5



In [ ]:
# # Dropping the quintuple system from the pairs catalog now...
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095859.91+130308.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095900.42+130241.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095903.29+130221.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095906.68+130135.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095908.96+130352.5']

# #Quadruples here
# xliu2011 = xliu2011[xliu2011['SDSS']!='J135632.71+232118.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J135635.73+232136.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J135638.42+232151.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J135639.45+232210.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J144702.08+134006.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J144705.78+133953.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J144708.49+133917.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J144712.81+133939.3']

# #Dropping all triples
# xliu2011 = xliu2011[xliu2011['SDSS']!='J075925.27+375216.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J075931.83+375252.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J075932.30+375319.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J082846.95+122724.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J082847.93+122832.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J082851.50+122726.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J085927.64+173344.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J085930.05+173409.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J085931.39+173253.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J090515.73+075618.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J090516.12+075522.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J090520.84+075440.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095942.61+261914.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095947.21+261711.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095948.15+261845.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J100722.96+230234.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J100725.21+230114.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J100732.36+230131.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102207.11+175017.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102212.61+175134.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102212.87+175113.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102223.90+154443.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102228.54+154514.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102228.76+154339.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102411.98+025234.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102415.08+025327.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102415.56+025241.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102546.26+134300.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102547.75+134456.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102548.60+134340.6']


# xliu2011 = xliu2011[xliu2011['SDSS']!='J105609.41+465341.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J105611.76+465250.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J105618.05+465155.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110441.92+041750.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110445.47+041755.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110449.56+041636.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110517.99+005328.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110520.67+005317.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110521.51+005256.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110719.72+025820.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110719.87+025926.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110723.69+025829.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J111005.18+032833.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J111005.60+032836.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J111007.99+032639.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J114938.17-033408.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J114939.73-033146.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J114949.79-033103.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120550.07+202837.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120551.47+203119.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120554.22+203421.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120718.36+204450.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120721.00+204503.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120722.37+204353.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121425.18+082251.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121428.15+082225.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121431.24+082225.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121442.50+125924.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121442.95+125746.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121443.11+125837.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J131628.81+525737.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J131629.82+525601.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J131637.18+525447.6']

# xliu2011 = xliu2011[xliu2011['SDSS']!='J132356.94+305917.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J132359.96+305556.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J132404.73+305723.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J133227.93+071859.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J133232.26+071739.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J133242.55+071938.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141057.24+252950.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141102.54+253109.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141105.14+252857.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141151.61-000303.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141153.47-000302.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141153.81-000203.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141421.15+025721.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141422.72+025834.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141422.99+025840.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143430.61+230529.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143438.51+230443.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143442.15+230637.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143801.02+101051.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143802.18+100944.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143810.34+100805.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150631.41+404146.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150640.58+404134.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150645.48+404051.5']

# xliu2011 = xliu2011[xliu2011['SDSS']!='J150821.57+191131.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150823.96+191229.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150831.30+191103.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151310.42+042855.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151311.70+042701.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151311.80+042952.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151640.00+215938.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151641.90+215955.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151644.79+215932.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160633.13+174632.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160638.60+174659.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160639.35+174540.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160709.88+183828.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160710.49+184004.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160715.79+184140.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J162906.60+114231.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J162907.29+114331.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J162908.94+114447.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J163300.85+114510.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J163304.69+114431.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J163305.33+114532.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231138.88+150741.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231142.06+150638.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231143.99+150603.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231224.52+135704.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231225.99+135450.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231229.22+135632.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J232402.98-095341.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J232403.39-095334.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J232407.99-095334.3']


